# Federated Learning
- Adapted from github given
- Need to modify svm function

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, '..')

In [4]:
from frecency import sample, frecency_points
from frecency import sample_suggestions_normal as sample_suggestions

## Linear Regression

This section is mostly to check that's it possible to fit a linear model perfectly to the data.

In [5]:
from sklearn.linear_model import LinearRegression

To fit a model, we sample a lot of these scores and also add noise on top to make the problem more similar to the real application.

In [6]:
n = int(1e6)
noise = np.random.normal(0, 2, size=(n))
X, y = sample(n)
y += noise

In [7]:
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

The resulting coefficients are extremely close to the actual frecency weights. How close they are depends on how much noise we add to the data matrix.

In [8]:
zip(model.coef_, frecency_points)

In [9]:
model.coef_ - frecency_points

array([ 0.01142331, -0.00414405, -0.00727568, -0.00296244,  0.00730539,
        0.00425576, -0.00085231,  0.00388006,  0.00033414,  0.01045079,
        0.00870776,  0.01075625, -0.0058682 ,  0.01195736,  0.00571056])

## Ranking and SVM loss

Now, we make the problem slightly more difficult: Instead of just learning the frecency function from data, we try to learn it from user interactions. The training data now consists of a variable number of history suggestions and their respective features. The label corresponds to the suggestion that the user clicked on. We still assume that the user clicks on the item with the highest frecency score.

### Preprocessing

Gradient descent generally works better when the data is centered around the origin:

In [10]:
from utils import normalize

In [11]:
X = normalize(X)

### Optimizers

In [12]:
from algos import GradientDescent, AdaptiveGradientDescent, DecayedGradientDescent, RProp, Adam

### Hinge Loss (SVM loss)

To supervise training, we keep logging the loss:

In [13]:
def svm_loss(preds, ys, delta=0):
    correct = ys.argmax()
    score_correct = preds[correct]
    
    loss = 0
    
    for i, pred in enumerate(preds):
        loss += max(0, pred + delta - score_correct)            
            
    return loss

### Helpers

During training, we want to supervise the learning process and save the best models.

In [14]:
from utils import ModelCheckpoint

accuracy needs to be measured carefully here: In our simulation, we assume that the current frecency is the perfect ranking function. But because items sometimes get the same frecency scores, there can be more than one correct answer:

In [15]:
def rank_accuracy(y, preds):
    correct = 0.
    
    for yi, pi in zip(y, preds):
        if yi[pi.argmax()] == yi.max():
            correct += 1
            
    return correct / len(y)

The `SVMRanking` class is the main mechanism for fitting models.

In [44]:
class SVMRanking:
    def __init__(self, delta):
        self.delta = delta
        
    def fit(self, data_generator, optimizer, num_iterations=10, callbacks=[]):
        X, y = data_generator(1)
        num_features = X[0].shape[1]
        self.W = frecency_points + (np.random.random(size=(num_features)) - 0.5) * 100
        
        for j in range(num_iterations):
            X, y = data_generator(4000)
            
            preds = self.predict(X)
            gradient = np.zeros(num_features)

            for xi, pi, yi in zip(X, preds, y):
                correct = yi.argmax()
                score_correct = pi[correct]

                for i, predicted_score in enumerate(pi):
                    gradient -= xi[i] * max(0, predicted_score + self.delta - score_correct)
            
            gradient /= len(X)
            
            loss = np.mean([svm_loss(pi, yi) for pi, yi in zip(self.predict(X), y)])
            accuracy = rank_accuracy(y, model.predict(X))
            
            print("[%d/%d] training: %.5f loss, %.3f accuracy" % (j + 1, num_iterations, loss, accuracy))
            
            for callback in callbacks:
                callback(self)
            
            self.W += optimizer(gradient)
            self.W -= 0.01 * 30 * self.W
    def predict(self, X):
        preds = []
        
        for x in X:
            scores = x.dot(self.W)
            preds.append(scores)
        
        return preds

### Training

In [45]:
np.random.seed(0)
model = SVMRanking(delta=0.)
model.fit(data_generator=sample_suggestions,
          optimizer=GradientDescent(30.),
          num_iterations=48,
          callbacks=[ModelCheckpoint(rank_accuracy, sample_suggestions)])

[1/48] training: 11.37885 loss, 0.704 accuracy
[ModelCheckpoint] New best model with 0.69890 validation accuracy
[2/48] training: 14.54165 loss, 0.699 accuracy
validation: 0.680 accuracy
[3/48] training: 9.38989 loss, 0.877 accuracy
[ModelCheckpoint] New best model with 0.88550 validation accuracy
[4/48] training: 0.60393 loss, 0.976 accuracy
[ModelCheckpoint] New best model with 0.97320 validation accuracy
[5/48] training: 0.28687 loss, 0.979 accuracy
[ModelCheckpoint] New best model with 0.97930 validation accuracy
[6/48] training: 0.12940 loss, 0.979 accuracy
[ModelCheckpoint] New best model with 0.98120 validation accuracy
[7/48] training: 0.05092 loss, 0.991 accuracy
[ModelCheckpoint] New best model with 0.99010 validation accuracy
[8/48] training: 0.03186 loss, 0.990 accuracy
validation: 0.988 accuracy
[9/48] training: 0.02373 loss, 0.987 accuracy
validation: 0.989 accuracy
[10/48] training: 0.01509 loss, 0.989 accuracy


KeyboardInterrupt: 

After fitting, we can compare the learned weights with the true frecency scores. Note that the values themselves are very different now but that the ordering is nearly the same as in the real algorithm. This shows that we are ranking very similarly to the real algorithm but that the optimization process did not fully reach the global optimum.